In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import math
import random
import pickle
import itertools

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, label_ranking_average_precision_score, label_ranking_loss, coverage_error 

from sklearn.utils import shuffle

from scipy.signal import resample

import matplotlib.pyplot as plt

np.random.seed(42)

import pickle
from sklearn.preprocessing import OneHotEncoder

from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Softmax, Add, Flatten, Activation# , Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score     #score evaluation package
from sklearn.model_selection import GridSearchCV     #classifier optimization search package

from sklearn.metrics import classification_report,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

from sklearn import svm

####### All Classifier packages #########
from sklearn.neighbors import KNeighborsClassifier    #KNN classifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier    # Decision Tree classifier
from sklearn.neural_network import MLPClassifier

####### Utilities ############
import numpy as np
import csv
# import emoji
from statistics import mean
import re
import collections
from textblob import TextBlob
from warnings import simplefilter
simplefilter(action='ignore', category=Warning)
# import en_core_web_sm
# import nltk 
# from nltk.corpus import wordnet 
# from  more_itertools import unique_everseen
# nltk.download('wordnet')
import pandas as pd
import tqdm

from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from datetime import datetime
import re
# import seaborn as sns
import collections


# Any results you write to the current directory are saved as output.
import numpy as np
from scipy import interp

from  sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer

def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [11]:
df = pd.read_csv("./data.csv", skiprows=0) # Training data, same format with test_data.csv
df = df.drop(columns=['Unnamed: 0'])
M = df.values
X = M[:, :-1]
y = M[:, -1].astype(int)
import pandas as pd
from sklearn import preprocessing

# x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
# df = pd.DataFrame(x_scaled)



In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify = y,random_state=2)


In [13]:
TreeModel = RandomForestClassifier( n_estimators = 100,
                                    criterion = "gini", 
                                    max_depth = None, 
                                    min_samples_split = 2,
                                    min_samples_leaf = 1, 
                                    max_features = "sqrt", 
                                    max_leaf_nodes= None, 
                                    class_weight = None, 
                                    bootstrap = True)
rf_model = TreeModel.fit(X_train, y_train)
y_predict = rf_model.predict(X_test)

# y_predict = rf_model.predict(X_test)

report_with_auc = class_report(y_true=y_test, y_pred=rf_model.predict(X_test), y_score=rf_model.predict_proba(X_test))
report_with_auc

precision    recall  f1-score  support   pred       AUC
0             0.621359  0.703297  0.659794     91.0  103.0  0.934815
1             0.666667  0.703297  0.684492     91.0   96.0  0.934138
2             0.912500  0.802198  0.853801     91.0   80.0  0.962009
3             0.936709  0.813187  0.870588     91.0   79.0  0.963555
4             0.865169  0.846154  0.855556     91.0   89.0  0.980256
5             0.919192  1.000000  0.957895     91.0   99.0  0.997633
avg / total   0.820266  0.811355  0.813688    546.0  546.0  0.969303